In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly=TRUE))
    install.packages("BiocManager")
BiocManager::install("splatter", build_vignettes=TRUE)

R[write to console]: Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.2 (2019-12-12)

R[write to console]: Installing package(s) 'BiocVersion', 'splatter'

R[write to console]: also installing the dependencies ‘rhdf5’, ‘Rhdf5lib’, ‘zlibbioc’, ‘bitops’, ‘lambda.r’, ‘futile.options’, ‘lsei’, ‘HDF5Array’, ‘RcppAnnoy’, ‘RcppHNSW’, ‘rsvd’, ‘XVector’, ‘RCurl’, ‘GenomeInfoDbData’, ‘sp’, ‘futile.logger’, ‘snow’, ‘limma’, ‘npsurv’, ‘DelayedArray’, ‘DelayedMatrixStats’, ‘BiocNeighbors’, ‘BiocSingular’, ‘beachmat’, ‘GenomicRanges’, ‘Biobase’, ‘IRanges’, ‘GenomeInfoDb’, ‘SingleCellExperiment’, ‘akima’, ‘BiocGenerics’, ‘BiocParallel’, ‘edgeR’, ‘fitdistrplus’, ‘locfit’, ‘matrixStats’, ‘scater’, ‘SummarizedExperiment’, ‘S4Vectors’


R[write to console]: trying URL 'https://bioconductor.org/packages/3.10/bioc/src/contrib/rhdf5_2.30.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 834814 bytes (815 KB)

R[write to console]: =
R[write to console]: =
R[

In [ ]:
%%R
library(splatter)
browseVignettes("splatter")